In [2]:
import psycopg2
from psycopg2 import OperationalError
import pandas as pd
import logging


In [9]:
ruta = 'C:\\Users\\anmmu\\OneDrive\\Proyectos\\999_Repositorios_Github\\MELI_app\\data\\data_matches - dataset.csv'
df_item = pd.read_csv(ruta, sep=',', encoding='utf-8')
df_item.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ITEM_A   28 non-null     int64 
 1   TITLE_A  28 non-null     object
 2   ITEM_B   28 non-null     int64 
 3   TITLE_B  28 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.0+ KB


In [11]:
df_items = pd.concat([df_item[['ITEM_A', 'TITLE_A']].rename(columns={'ITEM_A': 'ITEM', 'TITLE_A': 'TITLE'}), df_item[['ITEM_B', 'TITLE_B']].rename(columns={'ITEM_B': 'ITEM', 'TITLE_B': 'TITLE'})  ], ignore_index=True)
df_items.drop_duplicates(inplace=True)
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 0 to 41
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ITEM    39 non-null     int64 
 1   TITLE   39 non-null     object
dtypes: int64(1), object(1)
memory usage: 936.0+ bytes


In [15]:
# Establecer una nueva conexión
conn = psycopg2.connect(
    dbname="meli_app_db",
    user="user",
    password="password",
    host="localhost",
    port="5432"
)

# Crear un cursor para ejecutar la consulta
cursor = conn.cursor()


In [16]:
# Crear la tabla 'items' si no existe
cursor.execute("""
CREATE TABLE IF NOT EXISTS items (
    item_id BIGINT PRIMARY KEY,
    title TEXT
)
""")
conn.commit()

# Insertar registros en la tabla 'items'
for _, row in df_items.iterrows():
    cursor.execute(
        "INSERT INTO items (item_id, title) VALUES (%s, %s)",
        (row['ITEM'], row['TITLE'])
    )

# Confirmar los cambios en la base de datos
conn.commit()

In [17]:
try:
    
    # Consulta para obtener los nombres de las columnas de la tabla 'items'
    cursor.execute("""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_name = 'items';
    """)
    
    # Obtener los resultados
    headers = cursor.fetchall()
    print("Encabezados de la tabla 'items':", [header[0] for header in headers])


except OperationalError as e:
    print(f"Error: No se pudo conectar a la base de datos.")
    print(f"Detalle: {e}")

Encabezados de la tabla 'items': ['item_id', 'title']


In [18]:
try:
    # Consulta para obtener los primeros 50 registros de la tabla 'items'
    cursor.execute("""
        SELECT * 
        FROM items
        LIMIT 50;
    """)
    
    # Obtener los resultados
    records = cursor.fetchall()
    for record in records:
        print(record)

except OperationalError as e:
    print(f"Error: No se pudo conectar a la base de datos.")
    print(f"Detalle: {e}")


(514341, 'Baraja de Evangelion y Gundam W')
(687643, 'FAX TELEFONICA EQUIPOS - URGENTE')
(535665, 'autografo de Chilavert')
(2139803, 'Juego de tweeter.')
(2139565, 'Chomba color rojo.')
(468237168, 'Locales En Venta')
(63486251, 'Colección Vinilos Los Grandes Músicos')
(63486703, 'Zapato Y Botas De Mujer')
(63537003, 'Lote De 12pares Medias Soquetes 1/4 Toalla Algodon,no Envios')
(63480361, 'Lote X10 Excelentes Ponchos Toalla  Infantil Disney 60x120cm')
(63477742, 'Ch Africa Similar Edition 100 Ml - Tester De Free Shop')
(63482316, 'Entrada Bad Religion- Biohazard Obras 1993')
(634887722, 'Laser 4 Bocas Usado Para Reparar')
(63474586, 'Calefactor Placa Peabody Living & Arts Urgente')
(62476483, 'Gel De Armonicos')
(64263755, 'Trapos Rejilla De Algodón , 35 X 42 Cm De Pabilo-pack X 12')
(64263754, 'Medias Algodon, Mujer, Niño Y Niña Algodón')
(642875, 'Antigua Medalla Peronista Los Bombos De Peron Cgt Inusual!!')
(642765, 'Reloj De Bolsillo Perudi Swiss Agujas Azuladas Funciona Leer')


In [19]:
    
# Cerrar el cursor y la conexión
cursor.close()
conn.close()